# Домашнее задание по теме "Основы нейронных сетей"

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

import kaggle

## Загрузка данных

In [4]:
kaggle.api.authenticate()

In [16]:
kaggle.api.dataset_download_files(
    'vikrishnan/boston-house-prices',
    path='../data/raw/boston',
    unzip=True
)

Dataset URL: https://www.kaggle.com/datasets/vikrishnan/boston-house-prices


## Подготовка данных

In [18]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df = pd.read_csv('../data/raw/boston/housing.csv', delimiter=r"\s+", names=column_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


Пропущенных значений нет. Для загрузи данных в массив можно испльзовать `np.loadtxt` или `np.genfromtxt`

In [22]:
housing_numpy_01 = np.loadtxt('../data/raw/boston/housing.csv', dtype=np.float32)
housing_numpy_01.shape

(506, 14)

In [23]:
housing_numpy_02 = np.genfromtxt('../data/raw/boston/housing.csv', dtype=np.float32)
housing_numpy_02.shape

(506, 14)

### Создаем тензор

In [24]:
housing = torch.tensor(housing_numpy_01, dtype=torch.float32)
housing.shape

torch.Size([506, 14])

In [25]:
# Входные данные
inputs = housing[:, : -1]
inputs.shape

torch.Size([506, 13])

In [26]:
inputs_norm = torch.nn.functional.normalize(inputs)
inputs_norm.shape

torch.Size([506, 13])

In [27]:
# Выходные данные
labels = housing[:, -1]
labels.shape

torch.Size([506])

In [28]:
# Выровняем размерность
labels = labels.unsqueeze(1)
labels.shape

torch.Size([506, 1])

## Делим данные на train и val выборки

In [29]:
# Размер валидационной выборки
n_val = int(0.2 * inputs.shape[0])
n_val

101

In [30]:
# Перемешиваем индексы
shuffled_indeces = torch.randperm(inputs.shape[0])
# shuffled_indeces

In [31]:
train_indeces = shuffled_indeces[: -n_val]
val_indeces = shuffled_indeces[-n_val :]

In [32]:
train_inputs = inputs_norm[train_indeces]
train_labels = labels[train_indeces]

In [33]:
val_inputs = inputs_norm[val_indeces]
val_labels = labels[val_indeces]

In [34]:
# ds = TensorDataset(input, label)

In [389]:
# train_ds, val_ds = random_split(ds, [int(input.shape[0] * 0.8), input.shape[0] - (int(input.shape[0] * 0.8))])

In [390]:
# int(input.shape[0]) - int(input.shape[0] * 0.8)

In [35]:
train_ds = TensorDataset(train_inputs, train_labels)
val_ds = TensorDataset(val_inputs, val_labels)

In [36]:
batch_size = 100
train_iter = DataLoader(train_ds, batch_size, shuffle=True)

## Создание модели

In [37]:
model = torch.nn.Sequential(torch.nn.Linear(13, 1))
model

Sequential(
  (0): Linear(in_features=13, out_features=1, bias=True)
)

In [38]:
loss = torch.nn.MSELoss(reduction='mean')

In [39]:
trainer = torch.optim.Adam(model.parameters(), lr=0.001)

## Обучение модели

In [40]:
num_epoch = 10000
for epoch in range(1, num_epoch +1):
    for X, y in train_iter:
        trainer.zero_grad()
        l = loss(model(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model(train_inputs).reshape(-1), train_labels)
    if epoch % 5 == 0:
        print('Epoch %d, Loss: %f' % (epoch, l.item()),
              '|\tw', model[0].weight.data,
              '|\tb', model[0].bias.data)

/home/andrew/Projects/netology-pytorch-project/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/andrew/Projects/netology-pytorch-project/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/andrew/Projects/netology-pytorch-project/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([405, 1])) that is different to the input size (torch.Size([405])). T

Epoch 5, Loss: 591.969482 |	w tensor([[ 0.0486, -0.0085,  0.2271,  0.2171,  0.0383, -0.0251, -0.2118,  0.1701,
          0.0159, -0.0214, -0.2046,  0.0277,  0.1603]]) |	b tensor([-0.0070])
Epoch 10, Loss: 589.054688 |	w tensor([[ 0.0715,  0.0139,  0.2518,  0.2367,  0.0631, -0.0003, -0.1870,  0.1948,
          0.0405,  0.0034, -0.1798,  0.0525,  0.1852]]) |	b tensor([0.0179])
Epoch 15, Loss: 586.117126 |	w tensor([[ 0.0948,  0.0377,  0.2763,  0.2544,  0.0880,  0.0248, -0.1621,  0.2202,
          0.0651,  0.0284, -0.1546,  0.0777,  0.2102]]) |	b tensor([0.0430])
Epoch 20, Loss: 583.172607 |	w tensor([[ 0.1168,  0.0612,  0.3008,  0.2718,  0.1133,  0.0502, -0.1366,  0.2456,
          0.0897,  0.0535, -0.1292,  0.1030,  0.2357]]) |	b tensor([0.0682])
Epoch 25, Loss: 580.247070 |	w tensor([[ 0.1400,  0.0853,  0.3258,  0.2926,  0.1384,  0.0756, -0.1113,  0.2707,
          0.1142,  0.0784, -0.1038,  0.1282,  0.2606]]) |	b tensor([0.0933])
Epoch 30, Loss: 577.311890 |	w tensor([[ 0.1634,  0.108

## Проверка качества модели

In [43]:
val_iter = DataLoader(val_ds, batch_size=val_inputs.shape[0])
for X, y in val_iter:
        l = loss(model(X).reshape(-1), y)
        print(f"Validation loss {l}")
        

Validation loss 93.10267639160156
